In [2]:
!pip install bitsandbytes
!pip install transformers
!pip install peft
!pip install accelerate

In [6]:
from huggingface_hub import login
login()

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

# Setup quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized float 4
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model with quantization
model_id = "meta-llama/Llama-3.3-70B-Instruct"  # You'll need access to this
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# Define LoRA configuration
lora_config = LoraConfig(
    r=16,                  # Rank
    lora_alpha=32,         # Alpha parameter for LoRA scaling
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Which modules to apply LoRA to
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"  # Task type
)

# Apply LoRA adapters to model
model = get_peft_model(model, lora_config)

# Now you can train the model with your dataset using standard training loops
# The rest would involve setting up the dataset, training loop, etc.

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

model-00001-of-00030.safetensors:   0%|          | 0.00/4.58G [00:00<?, ?B/s]

model-00002-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

OSError: There was a specific connection error when trying to load meta-llama/Llama-3.3-70B-Instruct:
401 Client Error: Unauthorized for url: https://huggingface.co/meta-llama/Llama-3.3-70B-Instruct/resolve/main/model-00003-of-00030.safetensors (Request ID: Root=1-67cd8ff4-388f3f0c42df93da4e97d40d;7c83aec9-deac-45dd-9afc-7ac51979b5b8)

Invalid credentials in Authorization header